In [ ]:
!pip3 install vosk
!pip3 install sounddevice


In [ ]:
from vosk import Model, KaldiRecognizer
import sounddevice as sd
import queue
import json

# Setup
model = Model("./assets/vosk-model-small-en-us-0.15")  # path to the downloaded model
q = queue.Queue()

# Callback to store audio
def callback(indata, frames, time, status):
    if status:
        print(status)
    q.put(bytes(indata))

# Start listening
with sd.RawInputStream(samplerate=16000, blocksize=8000, dtype='int16',
                       channels=1, callback=callback):
    print("Say something...")
    rec = KaldiRecognizer(model, 16000)

    while True:
        data = q.get()
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            print("You said:", result.get("text", ""))
            break
